In [58]:
import pandas as pd
import numpy as np  
from scipy.stats import chi2_contingency
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [59]:
# 1. PREPARAR HOGARES (Para obtener V5 y REGION)
xl_hog = 'usu_hogar_T125.xlsx'
dfhog = pd.read_excel(xl_hog)
dfhog = dfhog.loc[:, ['CODUSU','NRO_HOGAR','DECIFR','REGION','V5_01','V5_02','V5_03']]

# Crear V5 (1=Percibe si alguno es 1, 2=No percibe)
dfhog['V5_RAW'] = 2 
mask_alguna_1 = (dfhog['V5_01'] == 1) | (dfhog['V5_02'] == 1) | (dfhog['V5_03'] == 1)
dfhog.loc[mask_alguna_1, 'V5_RAW'] = 1

# Recodificar y RENOMBRAR variables (Sin _CAT)
dfhog['V5'] = dfhog['V5_RAW'].replace({1: 'Percibe', 2: 'No Percibe'})
dfhog['REGION'] = dfhog['REGION'].replace({
    1: 'Gran Buenos Aires', 40: 'Noroeste', 41: 'Noreste', 
    42: 'Cuyo', 43: 'Pampeana', 44: 'Patagonia'
})

# Filtrar solo hogares pobres
dfhog_pobre = dfhog[dfhog['DECIFR'].isin([1, 2, 3, 4, 5])].copy()

# 2. PREPARAR PERSONAS (Variables de Control)
xl_usu = 'usu_individual_T125.xlsx'
dfusu = pd.read_excel(xl_usu)
# Importante: Cargar CH04 (Sexo), CH12 (Educ), CH06 (Edad)
dfusu = dfusu.loc[:, ['CODUSU','NRO_HOGAR','COMPONENTE','ESTADO','PONDERA','CH04','CH12','CH06']]

# Conversión numérica
dfusu['CH06'] = pd.to_numeric(dfusu['CH06'], errors='coerce').fillna(0)
dfusu['CH12'] = pd.to_numeric(dfusu['CH12'], errors='coerce').fillna(99)

# a) Lógica de Menores (Antes de filtrar)
dfusu['ES_MENOR'] = np.where(dfusu['CH06'] < 16, 1, 0)
# Marca al hogar si tiene al menos un menor
dfusu['HABITAN_MENORES'] = dfusu.groupby(['CODUSU', 'NRO_HOGAR'])['ES_MENOR'].transform('max')

# b) Filtrar Adultos (>=16) y Estado válido
df_adultos = dfusu[dfusu['CH06'] >= 16].copy()
df_adultos = df_adultos[df_adultos['ESTADO'].isin([1, 2, 3])] # Ocup, Desocup, Inactivo
df_adultos = df_adultos[df_adultos['CH12'] != 99]

# c) Recodificar Variables Personas
# ESTADO (Sin _CAT): Agrupamos 1 y 2 como Activo
df_adultos['ESTADO'] = df_adultos['ESTADO'].replace({1: 'Activo', 2: 'Activo', 3: 'Inactivo'})
# SEXO: Etiquetas claras
df_adultos['CH04'] = df_adultos['CH04'].replace({1: 'HOMBRES', 2: 'MUJERES'})
# EDUCACIÓN
df_adultos['NIVEL_ED_BIN'] = np.where(df_adultos['CH12'] >= 6, 'Alto', 'Bajo')

# 3. MERGE FINAL
df_final = dfhog_pobre.merge(df_adultos, on=['CODUSU', 'NRO_HOGAR'], how='inner')

# Ponderador Ajustado
df_final['PONDERA2'] = df_final['PONDERA'] * (len(df_final) / df_final['PONDERA'].sum())

print(f"Base lista. Variables finales: ESTADO, REGION, V5. Registros: {len(df_final)}")

Base lista. Variables finales: ESTADO, REGION, V5. Registros: 12424


In [60]:
# --- BLOQUE B: FUNCIÓN ESTADÍSTICA COMPLETA ---
from scipy.stats import chi2_contingency

def get_stats_results(abs_table):
    """Devuelve Chi2, V de Cramer y P-value correctamente formateados."""
    # Limpiar totales
    if 'Total' in abs_table.index: abs_table = abs_table.drop('Total', axis=0)
    if 'Total' in abs_table.columns: abs_table = abs_table.drop('Total', axis=1)
    
    if abs_table.empty or abs_table.sum().sum() == 0: return "Sin datos"
    
    try:
        # 1. Chi2 y P-value
        chi2, p_valor, dof, ex = chi2_contingency(abs_table)
        
        # 2. V de Cramer
        n = abs_table.sum().sum()
        min_dim = min(abs_table.shape) - 1
        if min_dim < 1: min_dim = 1
        cramer = np.sqrt(chi2 / (n * min_dim))
        
        # 3. Formateo Seguro (Evita notación científica tipo 1.2e-05)
        if p_valor < 0.0001:
            p_str = "<0.0001"
        else:
            p_str = f"{p_valor:.4f}"
            
        sig = "(*)" if p_valor < 0.05 else ""
        
        return f"Chi²={chi2:.2f}; V.Cramer={cramer:.3f}; p={p_str} {sig}"
        
    except Exception as e:
        return f"Error cálculo: {e}"

In [61]:
# --- BLOQUE C: LAZARSFELD (VARIABLES RENOMBRADAS) ---
print("Calculando tablas Lazarsfeld...")

# Definimos las variables con sus nombres NUEVOS
var_dep = 'ESTADO'
var_indep = 'V5'
var_control = 'REGION'

resultados_laz = [] # Para guardar y exportar luego

# 1. RELACIÓN ORIGINAL (V5 -> ESTADO)
t_orig = pd.pivot_table(df_final, values='PONDERA2', index=var_dep, columns=var_indep, aggfunc='sum', margins=True, margins_name='Total')
stats_orig = get_stats_results(t_orig)
t_orig_pct = t_orig.apply(lambda x: (x/x['Total'])*100).round(2)
df_orig_fin = pd.concat([t_orig, t_orig_pct], keys=['Absoluto', 'Porcentaje'])
resultados_laz.append((df_orig_fin, stats_orig, "RELACIÓN ORIGINAL (XY)"))

# 2. MARGINALES (REGION -> ESTADO y REGION -> V5)
# Marginal ZY (Region vs Estado)
t_zy = pd.pivot_table(df_final, values='PONDERA2', index=var_dep, columns=var_control, aggfunc='sum', margins=True, margins_name='Total')
stats_zy = get_stats_results(t_zy)
t_zy_pct = t_zy.apply(lambda x: (x/x['Total'])*100).round(2)
df_zy_fin = pd.concat([t_zy, t_zy_pct], keys=['Absoluto', 'Porcentaje'])
resultados_laz.append((df_zy_fin, stats_zy, "MARGINAL (ZY): Región vs Estado"))

# Marginal ZX (Region vs V5)
t_zx = pd.pivot_table(df_final, values='PONDERA2', index=var_indep, columns=var_control, aggfunc='sum', margins=True, margins_name='Total')
stats_zx = get_stats_results(t_zx)
t_zx_pct = t_zx.apply(lambda x: (x/x['Total'])*100).round(2)
df_zx_fin = pd.concat([t_zx, t_zx_pct], keys=['Absoluto', 'Porcentaje'])
resultados_laz.append((df_zx_fin, stats_zx, "MARGINAL (ZX): Región vs V5"))

# 3. PARCIALES (XY controlando por Región)
regiones = sorted(df_final[var_control].unique())
for reg in regiones:
    df_r = df_final[df_final[var_control] == reg]
    if df_r.empty: continue
    
    t_par = pd.pivot_table(df_r, values='PONDERA2', index=var_dep, columns=var_indep, aggfunc='sum', margins=True, margins_name='Total')
    stats_par = get_stats_results(t_par)
    t_par_pct = t_par.apply(lambda x: (x/x['Total'])*100).round(2)
    df_par_fin = pd.concat([t_par, t_par_pct], keys=['Absoluto', 'Porcentaje'])
    resultados_laz.append((df_par_fin, stats_par, f"PARCIAL: {reg}"))

print("Cálculos de Lazarsfeld terminados. Listo para exportar.")

Calculando tablas Lazarsfeld...
Cálculos de Lazarsfeld terminados. Listo para exportar.


In [62]:
# --- BLOQUE D: MODELOS DE REGRESIÓN (ORDENADO Y LIMPIO) ---
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pandas as pd
import numpy as np

print("Calculando Modelos de Regresión...")

# 1. Preparar Variable Dependiente
df_final['ESTADO_BIN'] = np.where(df_final['ESTADO'] == 'Activo', 1, 0)

# 2. Funciones Auxiliares
def get_nagelkerke_r2(model_res):
    """Calcula Pseudo R2"""
    n = model_res.nobs
    llf = model_res.llf
    lln = model_res.llnull
    r2_cs = 1 - np.exp((2/n) * (lln - llf))
    r2_max = 1 - np.exp((2/n) * lln)
    return r2_cs / r2_max if r2_max > 0 else 0

def format_regression_output(modelo, nombre_modelo, estrato, n_ponderado_real):
    """Extrae coeficientes y formatea"""
    df_res = pd.DataFrame({
        'Modelo': nombre_modelo,
        'Estrato': estrato,
        'Coef (Beta)': modelo.params,
        'Odds Ratio': np.exp(modelo.params),
        'Error Std': modelo.bse,
        'P-value': modelo.pvalues
    })
    
    # Formateo estricto de P-Value
    df_res['P-value'] = df_res['P-value'].apply(lambda x: "<0.0001" if x < 0.0001 else f"{x:.4f}")
    
    # Redondeo
    df_res['Coef (Beta)'] = df_res['Coef (Beta)'].round(4)
    df_res['Odds Ratio'] = df_res['Odds Ratio'].round(4)
    df_res['Error Std'] = df_res['Error Std'].round(4)
    
    # Métricas
    r2 = get_nagelkerke_r2(modelo)
    df_res['Nagelkerke R2'] = f"{r2:.4f}"
    df_res['N (Ponderado)'] = round(n_ponderado_real, 2)
    
    # LIMPIEZA DE ETIQUETAS (Reset index mueve los nombres de variables a una columna)
    df_out = df_res.reset_index().rename(columns={'index': 'Variable'})
    
    # Quitamos el texto largo de Treatment(...) para que quede limpio
    # Ejemplo: C(REGION, Treatment(reference='Gran Buenos Aires'))[T.Noreste] -> C(REGION)[T.Noreste]
    df_out['Variable'] = df_out['Variable'].astype(str).str.replace(r"C\(REGION, Treatment\(reference='.*?'\)\)", "C(REGION)", regex=True)
    
    return df_out

all_reg_results = []

# =============================================================================
# 1. REGRESIÓN MÚLTIPLE NACIONAL (La Controlada)
# =============================================================================
print("  > Ajustando Modelo Nacional Múltiple...")
try:
    # Usamos Gran Buenos Aires como referencia explícita
    formula_nac = "ESTADO_BIN ~ C(V5) + C(REGION, Treatment(reference='Gran Buenos Aires')) + C(CH04) + C(NIVEL_ED_BIN) + C(HABITAN_MENORES)"
    
    mod_nac = smf.glm(formula=formula_nac, data=df_final, 
                      family=sm.families.Binomial(), 
                      freq_weights=df_final['PONDERA2']).fit()
    
    n_real_nac = df_final['PONDERA2'].sum()
    df_nac = format_regression_output(mod_nac, "Logística Múltiple", "Total País", n_real_nac)
    all_reg_results.append(df_nac)
    
except Exception as e:
    print(f"Error en Nacional Múltiple: {e}")

# =============================================================================
# 2. REGRESIONES SIMPLES (Todas Juntas)
# =============================================================================

# A) Simple Nacional
print("  > Ajustando Modelo Nacional Simple...")
try:
    formula_simple_nac = 'ESTADO_BIN ~ C(V5)'
    mod_sim_nac = smf.glm(formula=formula_simple_nac, data=df_final, 
                          family=sm.families.Binomial(), 
                          freq_weights=df_final['PONDERA2']).fit()
    
    n_real = df_final['PONDERA2'].sum()
    # Quitamos "(Base)" del nombre como pediste
    df_sim_nac = format_regression_output(mod_sim_nac, "Logística Simple", "Total País", n_real)
    all_reg_results.append(df_sim_nac)
except Exception as e:
    print(f"Error en Simple Nacional: {e}")

# B) Simples Regionales
print("  > Ajustando Modelos Regionales Simples...")
regiones = sorted(df_final['REGION'].unique())
formula_simple = 'ESTADO_BIN ~ C(V5)' 

for reg in regiones:
    df_reg = df_final[df_final['REGION'] == reg]
    if df_reg['ESTADO_BIN'].nunique() < 2: continue 
    
    try:
        mod_reg = smf.glm(formula=formula_simple, data=df_reg,
                          family=sm.families.Binomial(),
                          freq_weights=df_reg['PONDERA2']).fit()
        
        n_real_reg = df_reg['PONDERA2'].sum()
        df_res_reg = format_regression_output(mod_reg, "Logística Simple", reg, n_real_reg)
        all_reg_results.append(df_res_reg)
        
    except Exception as e:
        print(f"Error en región {reg}: {e}")

# =============================================================================
# CONSOLIDACIÓN
# =============================================================================
if all_reg_results:
    df_reg_final = pd.concat(all_reg_results, ignore_index=True)
    
    # Orden final de columnas
    cols = ['Modelo', 'Estrato', 'Variable', 'Coef (Beta)', 'Odds Ratio', 'Error Std', 'P-value', 'Nagelkerke R2', 'N (Ponderado)']
    df_reg_final = df_reg_final[cols]
    
    print("\nCálculos Completados.")
    print("Orden: Múltiple -> Simple Nacional -> Simples Regionales")
    display(df_reg_final.head(15)) # Mostramos más filas para ver el salto de Múltiple a Simple
else:
    print("No se generaron modelos.")

Calculando Modelos de Regresión...
  > Ajustando Modelo Nacional Múltiple...
  > Ajustando Modelo Nacional Simple...
  > Ajustando Modelos Regionales Simples...

Cálculos Completados.
Orden: Múltiple -> Simple Nacional -> Simples Regionales


,Modelo,Estrato,Variable,Coef (Beta),Odds Ratio,Error Std,P-value,Nagelkerke R2,N (Ponderado)
0,Logística Múltiple,Total País,Intercept,0.6674,1.9492,0.0504,<0.0001,0.1133,12424.00
1,Logística Múltiple,Total País,C(V5)[T.Percibe],0.2168,1.2421,0.0523,<0.0001,0.1133,12424.00
2,Logística Múltiple,Total País,C(REGION)[T.Cuyo],0.0279,1.0283,0.0748,0.7091,0.1133,12424.00
3,Logística Múltiple,Total País,C(REGION)[T.Noreste],-0.2161,0.8056,0.0795,0.0066,0.1133,12424.00
4,Logística Múltiple,Total País,C(REGION)[T.Noroeste],-0.0802,0.9230,0.0591,0.1748,0.1133,12424.00
5,Logística Múltiple,Total País,C(REGION)[T.Pampeana],-0.0819,0.9214,0.0476,0.0852,0.1133,12424.00
6,Logística Múltiple,Total País,C(REGION)[T.Patagonia],-0.1290,0.8790,0.1112,0.2458,0.1133,12424.00
7,Logística Múltiple,Total País,C(CH04)[T.MUJERES],-0.9791,0.3756,0.0389,<0.0001,0.1133,12424.00
8,Logística Múltiple,Total País,C(NIVEL_ED_BIN)[T.Bajo],-0.4548,0.6346,0.0443,<0.0001,0.1133,12424.00
9,Logística Múltiple,Total País,C(HABITAN_MENORES)[T.1],0.7663,2.1518,0.0455,<0.0001,0.1133,12424.00


In [63]:
# --- BLOQUE E: EXPORTACIÓN FINAL A EXCEL ---
print("Iniciando guardado de Excel...")

output_filename = 'Output_EPH_Lazarsfeld_Regresion.xlsx'

try:
    writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')
    
    # --- PESTAÑA 1: LAZARSFELD ---
    sheet_laz = 'Lazarsfeld_Tables'
    row = 0
    
    if 'resultados_laz' in locals() and resultados_laz:
        for df_table, stats_str, titulo in resultados_laz:
            # Título
            pd.DataFrame([titulo]).to_excel(writer, sheet_name=sheet_laz, startrow=row, header=False, index=False)
            row += 1
            # Tabla
            df_table.to_excel(writer, sheet_name=sheet_laz, startrow=row)
            # Estadísticos
            pd.DataFrame([stats_str]).to_excel(writer, sheet_name=sheet_laz, startrow=row, startcol=10, header=False, index=False)
            row += df_table.shape[0] + 4
        print(f"Pestaña '{sheet_laz}' generada.")

    # --- PESTAÑA 2: REGRESIONES (CONSOLIDADA) ---
    sheet_reg = 'Regresion_Resultados'
    if 'df_reg_final' in locals() and not df_reg_final.empty:
        # Simplemente volcamos la tabla maestra que creamos en el Bloque D
        df_reg_final.to_excel(writer, sheet_name=sheet_reg, index=False)
        print(f"Pestaña '{sheet_reg}' generada con modelo Nacional y Regionales.")
    else:
        print("Advertencia: No hay tabla de regresión para guardar.")

    writer.close()
    print(f"\n¡ÉXITO! Archivo '{output_filename}' guardado correctamente.")

except Exception as e:
    print(f"Error crítico al guardar Excel: {e}")

Iniciando guardado de Excel...
Pestaña 'Lazarsfeld_Tables' generada.
Pestaña 'Regresion_Resultados' generada con modelo Nacional y Regionales.

¡ÉXITO! Archivo 'Output_EPH_Lazarsfeld_Regresion.xlsx' guardado correctamente.
